In [0]:
%sql
/* 03_gold_claims_by_hour.ipynb
GOAL 1: Hourly claim volume and spend (count, sum, avg)
GOAL 2: Daily provider spend with 7-day rolling totals (window function demo)
GOAL 3: QA view — hourly provider match rate

SOURCE: kardia_silver.silver_claims_enriched
OUTPUT: TABLE: gold_hourly_claim_metrics
        TABLE: gold_provider_7d_spend
        VIEW : gold_hourly_match_qc_vw

TRIGGER: Full snapshot overwrite each run — fast and simple for small datasets.
         Use foreachBatch and MERGE in production to avoid rewriting unchanged data.

NOTE: The Gold Claims/Providers tables use Delta Liquid Clustering and Predictive Optimization
      to reduce small files and maintain layout efficiency without manual OPTIMIZE or ZORDER. */

USE kardia_gold;

-- Config
SET kflow.claims.hourly.time_col = _ingest_ts;

In [0]:
%sql
-- 2. Create hourly claim metrics table (CTAS)
CREATE OR REPLACE TABLE gold_hourly_claim_metrics AS
WITH base AS (
  SELECT
    DATE_TRUNC('HOUR', ${kflow.claims.hourly.time_col}) AS hour_ts,
    claim_amount,
    provider_specialty
  FROM kardia_silver.silver_claims_enriched
)
SELECT
  hour_ts,
  COUNT(*)                              AS claim_cnt,
  SUM(claim_amount)                     AS total_amount,
  AVG(claim_amount)                     AS avg_amount,
  SUM(CASE WHEN provider_specialty IS NULL THEN 1 ELSE 0 END) AS unmatched_cnt
FROM base
GROUP BY hour_ts;

In [0]:
%sql
/* Define a QA view that calculates the hourly provider match rate
   by dividing matched claims by total claims from the hourly metrics. */
   
CREATE OR REPLACE VIEW gold_hourly_match_qc_vw AS
SELECT
  hour_ts,
  claim_cnt,
  unmatched_cnt,
  ROUND(1 - unmatched_cnt / claim_cnt, 4) AS match_rate
FROM gold_hourly_claim_metrics;


In [0]:
%sql
-- Preview: Top 10 rows from hourly claim metrics
SELECT * FROM gold_hourly_claim_metrics
ORDER BY hour_ts DESC
LIMIT 10;

In [0]:
%sql
-- Preview: Top 10 rows from provider 7-day spend
SELECT * FROM gold_provider_7d_spend
ORDER BY claim_date DESC, provider_id
LIMIT 10;

In [0]:
%sql
-- Preview: Top 10 rows from hourly provider match QA view
SELECT * FROM gold_hourly_match_qc_vw
ORDER BY hour_ts DESC
LIMIT 10;